### **1. Importing necessary libraries**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import keras
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from utils.utils import plot_confusion_matrix, split_data

### **2. CNN Model**

In [2]:
# loading model
model = keras.models.load_model('./models/trained/cnn_model.h5')

model.summary()

In [3]:
# loading model's history
with open('./models/historys/cnn_model_history', 'rb') as f:
    history = pickle.load(f)

#### 2.1 Curves

In [4]:
plt.figure(figsize = (10, 4))

plt.subplot(1, 2, 1)
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Training", "Accuracy"])

plt.subplot(1, 2, 2)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Training Loss", "Validation Loss"])

plt.savefig('./models/figures/cnn/CNN-curves.jpg')

plt.show()

#### 2.2 Loading datas

In [5]:
#load dataset
dataset = pd.read_csv('../dataset/dataset_final.csv')

# refactor values in filepath column added ../ to the path
dataset['Filepath'] = dataset['Filepath'].apply(lambda x: '../' + x)

dataset.head()

In [6]:
train_df, test_df = train_test_split(
    dataset.sample(frac=1.0, random_state=42),
    train_size=0.80,
    random_state=42
)

In [7]:
train_data, validation_data, test_data = split_data(train_df, test_df)

#### 2.3 Testing the model

In [8]:
y_pred = np.squeeze(model.predict(test_data) >= 0.5).astype(np.int32)

In [9]:
from sklearn.metrics import f1_score, precision_score, recall_score

x = f1_score(test_data.labels, y_pred)
y = precision_score(test_data.labels, y_pred)
z = recall_score(test_data.labels, y_pred)

In [10]:
print(f'F1 Score: {round(x*100,2)}%')
print(f'Precision Score: {round(y*100,2)}%')
print(f'Recall Score: {round(z*100,2)}%')

In [11]:
results = model.evaluate(test_data)

# assigning the results into loss and accuracy
loss = results[0]
accuracy = results[1]

# showing up the results
print(f"Model's accuracy: {(accuracy*100):0.2f}%")
print(f"Model's loss: {(loss):0.2f}")

In [12]:
# creating the confusion matrix
classifications = classification_report(test_data.labels, y_pred, target_names = ["Without crack", "With crack"])

print(classifications)

# ploting the Matrix
disp = plot_confusion_matrix(test_data.labels, y_pred, title="CNN")

In [13]:
disp.figure_.savefig('./models/figures/cnn/CNN-confusion-matrix_final.png')
disp.figure_.savefig('./models/figures/cnn/CNN-confusion-matrix_final.eps')